In [1]:
import os, sys
import ROOT
sys.path.append("../Utils/")
sys.path.append("../Config/")
import xconfig
import xutils
from fillReport import *
import math

fillReport = FillReport('../Config/FillReport_1446656923991.xls')
#
cfg = xconfig.Config('../Config/sourceCalibration.ini')
cfgData = cfg.get_option('Detectors')
detInd = {}
detNo = {}
detCf = {}
detCfErr = {}
for key in cfgData.keys():
    tmp = cfgData[key].split()
    detInd[key] = int(tmp[0])
    detNo[key] = int(tmp[1])
    detCf[key] = float(tmp[2])
    detCfErr[key] = float(tmp[3])
del cfg

In [2]:
inputPattern = '/home/data/RadmonLumi/radmonLumi__XXX__.root'
refIndx = 5   # PFIB used to find beginning of the collisions
rateCut = 50.  # If rate higher than this value, collisions started 
minPeakInstLumi = 2000.  # Only fills with larger lumi are taken
deltaT = 2000.  #  Warm-up cut  
minBfield = 3.6

#fills = [ 4528, 4530]
fills = [ 3960, 3962, 3965, 3971, 3974,3976, 3981, 3983, 3986, 3988, 3992, 3996, 
        4001, 4006, 4008, 4019, 4020, 4201, 4205, 4207, 4208, 4210, 4211, 4212, 
         4214, 4219, 4220, 4224, 4225, 4231, 4243, 4246, 4249, 4254, 4256, 4257, 
         4266, 4268, 4269, 4322, 4323, 4332, 4337, 4341, 4342, 4349, 4356, 4360, 
         4363, 4364, 4376, 4381, 4384, 4386, 4391, 4393, 4397, 4398, 4402, 4410, 
         4418, 4420, 4423, 4426, 4428, 4432, 4434, 4435, 4437,  4466, 4467,4476, 4477, 4479, 4485, 
         4495, 4496, 4499, 4505, 4509, 4510, 4511, 4513, 4518, 4519, 4522, 4525, 
         4528, 4530, 4532, 4536, 4538, 4540, 4545, 4555, 4557, 4562, 4565, ]
#         4440, 4444, 4448, 4449, 4452, 4455, 4462, 4463, 4464,]
#fills = [4440, 4444, 4448, 4449, 4452, 4455, 4462, 4463, 4464,]

In [4]:
graphs = {}
for key in detInd.keys():
    graphs[key] = ROOT.TGraph()
    graphs[key].SetTitle(key.upper() + " Neutron fluence vs bestLumi ")

print "Starting..."
for fill in fills:
    
    # Fill cuts
    if fillReport.getFillPeakInstLumi(fill) < minPeakInstLumi:
        #print "Low FillPeakInstLumi for fill", fill
        continue
    if fillReport.getFillField(fill) < minBfield:
        #print "Low magnetic field for fill", fill
        continue       
    if fillReport.getFillDuration(fill) < deltaT:
        #print "Short fill", fill
        continue
    
    inputFile = inputPattern.replace('__XXX__', str(fill))
    try:
        fin = ROOT.TFile(inputFile,'READ')
        #print "Processing fill", fill
    except IOError:
        print "Cannot open file", inputFile
        continue
    t = fin.Get("t")  
    
    tsColl = -10000
    for i in range(0, t.GetEntries()):
        nb = t.GetEntry(i)
        if nb < 0:
            continue
        
        #Start of collisions 
        if tsColl < 0:
            if t.rates[refIndx] > rateCut:
                tsColl = t.tstamp  
        if tsColl < 0 or t.tstamp < tsColl + deltaT:
            continue

        for key in graphs.keys():
            n = graphs[key].GetN()
            graphs[key].SetPoint(n, t.bestLumi, detCf[key]*t.rates[detInd[key]])
            #graphs[key].SetPoint(n, t.bestLumi, t.rates[detInd[key]])

            
print "Done" 

Starting...
Done


##Fit graphs

In [5]:
func = ROOT.TF1('func', '[0]+[1]*x', minPeakInstLumi/2., 6000.)
func.SetParameters(0., 3.)
for key in graphs.keys():
    #func.SetParameter(0, 0.)
    #func.FixParameter(0, 0.)
    graphs[key].Fit('func', "CROBFQ", "", minPeakInstLumi/2., 6000.)

## Print fluence

In [6]:
p0 ={}
dp0 = {}
p1 = {}
dp1 = {}
for key in graphs.keys():
    p0[key] = graphs[key].GetFunction('func').GetParameter(0)
    dp0[key] = graphs[key].GetFunction('func').GetParError(0)
    p1[key] = graphs[key].GetFunction('func').GetParameter(1)
    dp1[key] = graphs[key].GetFunction('func').GetParError(1)    

for key in graphs.keys():
    print key.upper(), ':\t', '%.3f' %p1[key]  #, '+/-', dp1[key]

MFXT :	0.265
MNIT :	3.165
PFIB :	4.982
PNIT :	3.327
MFIT :	0.476
MNIB :	4.196
PFIT :	3.267
MFIB :	3.078
MNXT :	0.250
PFXT :	0.280
PNIB :	4.072


## Print ratios 

In [7]:
ratios = {'MFIB/MNIB' , 'MFIT/MFIB', 'MFIT/MNIT', 'MNIT/MNIB', 'PFIB/PNIB', 'PFIT/PFIB', 'PFIT/PNIT', 'PNIT/PNIB' }
for key in ratios:
    (nom, denom) = key.split('/')
    value = p1[nom.lower()]/p1[denom.lower()]
    print key, '\t', '%.3f' % value

MFIT/MNIT 	0.150
PNIT/PNIB 	0.817
MNIT/MNIB 	0.754
PFIT/PNIT 	0.982
PFIB/PNIB 	1.223
PFIT/PFIB 	0.656
MFIT/MFIB 	0.155
MFIB/MNIB 	0.733


In [6]:
c = ROOT.TCanvas()
ROOT.gStyle.SetOptFit(1111)
key = 'pnib'
print detCf[key]
graphs[key].Draw('AP')

1.75
